In [1]:
%matplotlib inline
import numpy as np
import theano.tensor as tt
import pymc3 as pm

import seaborn as sns
import matplotlib.pyplot as plt

sns.set_context('notebook')
plt.style.use('seaborn-darkgrid')
print('Running on PyMC3 v{}'.format(pm.__version__))

Running on PyMC3 v3.8


In [20]:
import pandas as pd
# data format: [all time right, all time wrong, time elapsed in minutes, number of sub-quizzes, sub-quizzes right]
data = pd.DataFrame(np.array([[1, 0, 12, 2,2.], [1,1, 10., 1, 1], [5, 1, 400., 3, 2], [4,2,99., 2,2], [19, 4,1931., 3,3]]), columns=['right', 'wrong', 'elapsed', 'nquiz', 'nright'])
data['sqrtright']=np.sqrt(1+data.right)
data['sqrtwrong']=np.sqrt(1+data.wrong)
data

,right,wrong,elapsed,nquiz,nright,sqrtright,sqrtwrong
0,1.0,0.0,12.0,2.0,2.0,1.414214,1.000000
1,1.0,1.0,10.0,1.0,1.0,1.414214,1.414214
2,5.0,1.0,400.0,3.0,2.0,2.449490,1.414214
3,4.0,2.0,99.0,2.0,2.0,2.236068,1.732051
4,19.0,4.0,1931.0,3.0,3.0,4.472136,2.236068


In [39]:
million = pd.read_csv("/Users/fasih/Dropbox/src/halflife-regression/pymcmill.csv")
million['sqrtright']=np.sqrt(1+million.history_correct)
million['sqrtwrong']=np.sqrt(1+(million.history_seen - million.history_correct))
million = million[:1000]
million

,delta,history_seen,history_correct,session_seen,session_correct,sqrtright,sqrtwrong
0,3065627,3,3,2,2,2.000000,1.000000
1,335,266,256,2,2,16.031220,3.316625
2,481,5,5,1,0,2.449490,1.000000
3,336093,18,16,3,3,4.123106,1.732051
4,411,150,136,1,1,11.704700,3.872983
...,...,...,...,...,...,...,...
995,1366,6,6,1,1,2.645751,1.000000
996,1806,3,3,4,4,2.000000,1.000000
997,338,8,8,5,5,3.000000,1.000000
998,14356,3,3,1,1,2.000000,1.000000


In [ ]:
np.sqrt(150)

In [ ]:
with pm.Model() as model:
    # hyperpriors
    hyperpriorMu = pm.Normal('hyperpriorMu', mu=0, sigma=5)
    hyperpriorStd = pm.Uniform('hyperpriorStd', 1, 30) # pm.HalfCauchy('hyperpriorStd', beta=3)
    
    # weights
    walpha = pm.Normal('walpha', mu=hyperpriorMu, sigma=hyperpriorStd, shape=3)
    wbeta = pm.Normal('wbeta', mu=hyperpriorMu, sigma=hyperpriorStd, shape=3)
    
    # Ebisu-style variables
    alpha = (walpha[0] * million.sqrtright.values + walpha[1] * million.sqrtwrong.values + walpha[2]).clip(1, 100)
    beta = (wbeta[0] * million.sqrtright.values + wbeta[1] * million.sqrtwrong.values + wbeta[2]).clip(1, 100)

    p = pm.Beta('p', alpha=alpha, beta=beta, shape=len(million), testval=0.7)
    x = pm.Binomial('x', n=million.session_seen.values, p=p, observed=million.session_correct.values)
    
    trace = pm.sample(5000, tune=1000, cores=2, progressbar=True)
pm.traceplot(trace);

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [p, wbeta, walpha, hyperpriorStd, hyperpriorMu]
Sampling 2 chains, 0 divergences:  18%|█▊        | 2217/12000 [03:43<25:29,  6.40draws/s]  